In [27]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf 
import tensorflow_io as tfio
from os import listdir
from os.path import isfile, join
import numpy as np  
import librosa
import librosa.display
import pandas as pd 
import matplotlib.pyplot as plt
from pathlib import Path

In [28]:
def replace_backslash_with_forwardslash(filenames):
    return [elem.replace('\\', '/') for elem in filenames]


In [29]:
# mypath = os.path.join('RespiratorySoundDatabase','Respiratory_Sound_Database','Respiratory_Sound_Database','audio_and_txt_files')
mypath = './RespiratorySoundDatabase/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/'
filenames = [f for f in listdir(mypath) if (isfile(join(mypath, f)) and f.endswith('.wav'))] 
print(mypath)

./RespiratorySoundDatabase/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/


In [30]:
p_id_in_file = [] # patient IDs corresponding to each file
for name in filenames:
    p_id_in_file.append(int(name[:3]))

p_id_in_file = np.array(p_id_in_file)

In [31]:
max_pad_len = 862 # to make the length of all MFCC equal

def extract_features(file_name):
    """
    This function takes in the path for an audio file as a string, loads it, and returns the MFCC
    of the audio"""
   
    try:
        # audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast', duration=20) 
        # mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        audio, sample_rate = librosa.load(file_name)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    # pad_width = max_pad_len - mfccs.shape[1]
    # mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
    return mfccs

   
# with open('RespiratorySoundDatabase\Respiratory_Sound_Database\Respiratory_Sound_Database\audio_and_txt_files\104_1b1_Al_sc_Litt3200.wav','w'):        
    # extract_features(rf'RespiratorySoundDatabase\Respiratory_Sound_Database\Respiratory_Sound_Database\audio_and_txt_files\104_1b1_Al_sc_Litt3200.wav')
extract_features(rf'RespiratorySoundDatabase\Respiratory_Sound_Database\Respiratory_Sound_Database\audio_and_txt_files\104_1b1_Ll_sc_Litt3200.wav')

array([[-4.64041199e+02, -4.18290039e+02, -3.87566589e+02, ...,
        -4.88370483e+02, -4.62626923e+02, -4.20243561e+02],
       [ 8.44875793e+01,  1.36891495e+02,  1.71178497e+02, ...,
         5.40320740e+01,  8.63839722e+01,  1.16796173e+02],
       [ 6.75648041e+01,  8.99741745e+01,  1.04197815e+02, ...,
         4.75569954e+01,  6.90976944e+01,  4.86898804e+01],
       ...,
       [-2.82355607e-01,  2.05963564e+00,  1.22030616e+00, ...,
        -5.73893607e-01,  4.95867699e-01,  2.84204936e+00],
       [-6.52661860e-01,  1.89648938e+00,  5.53389192e-01, ...,
        -9.39450562e-01, -2.31588945e-01,  1.73905706e+00],
       [-6.38449073e-01,  7.99845815e-01, -1.81964302e+00, ...,
        -1.14678538e+00, -5.82178056e-01,  2.36425489e-01]], dtype=float32)

In [32]:
filepaths = [join(mypath, f) for f in filenames] # full paths of files

# for f in filepaths:
#     print(f)

In [33]:
p_diag = pd.read_csv("./RespiratorySoundDatabase/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv",header=None) # patient diagnosis file
# print(p_diag)

In [34]:
labels = np.array([p_diag[p_diag[0] == x][1].values[0] for x in p_id_in_file]) # labels for audio files
# print(labels)

In [38]:
from soundfile import SoundFile
features = [] 

# Iterate through each sound file and extract the features
for file_name in filepaths:
    with SoundFile(file_name, 'r') as file_name:
        data = extract_features(file_name)
        features.append(data)

print('Finished feature extraction from ', len(features), ' files')
# features = np.array(features,dtype=object).flatten() 
for i in features:
    print(i)

Finished feature extraction from  920  files
[[-2.8113452e+02 -3.4665237e+02 -5.2626855e+02 ... -5.2192291e+02
  -4.0611319e+02 -2.8902808e+02]
 [ 1.2077900e+02  1.2382669e+02  1.0786206e+02 ...  1.1370058e+02
   1.2294359e+02  1.0859164e+02]
 [ 1.5145942e+01  1.8448498e+01  7.0185539e+01 ...  7.7038658e+01
   4.1171021e+01  2.8062628e+01]
 ...
 [ 2.0264161e+00  2.3279653e+00  3.4543818e-01 ... -6.9325142e+00
  -1.0097846e+00  4.4193655e-01]
 [ 6.7942929e-01  1.4032114e+00  2.3228798e+00 ... -2.2480469e+00
  -1.0891582e+00  2.2538124e-01]
 [ 2.5556307e+00  3.0634472e+00  4.0933304e+00 ...  3.1338601e+00
  -4.7370979e-01  5.5637550e-01]]
[[-3.7470538e+02 -4.3860428e+02 -5.7013251e+02 ... -4.9728439e+02
  -4.3360965e+02 -3.4630835e+02]
 [ 1.1936230e+02  1.2258203e+02  1.0890018e+02 ...  1.7822940e+02
   1.5959219e+02  1.1511362e+02]
 [ 1.3773283e+01  1.7932940e+01  6.1386276e+01 ...  6.7255264e+01
   5.2953064e+01  3.2698475e+01]
 ...
 [ 2.2594419e+00  1.0125852e+00  1.9362919e+00 ...  2